# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=WN0z8Fz42ZtajOnr23RuBp3tsR4joo&prompt=consent&token_usage=remote&access_type=offline&code_challenge=hRZ18DlXMnJS5R9DFno9pt8WbwRiK-bEURg4SGRmECo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJiioePAnOLZxp4KD4jv-cWSdxL_YwjyknHP8dowzFcCRMhqZZ4RZrYb1gRPjjFlDA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

# Tratamento

Foi feito o download do mês de novembro da remuneração dos aposentados e pensionistas civis e reserva/reforma militares

In [ ]:
# Import necessary libraries
from google.cloud import bigquery
import pandas as pd

# Initialize the BigQuery client, specifying the Google Cloud project ID.
client = bigquery.Client(project='repositoriodedadosgpsp')
# Define a list of years to process.
anos = [2020,2021,2022,2023,2024]
# Loop through each year in the list.
for ano in anos:
  # Define a formatted SQL query to be executed for each year.
  query = f"""
  # Use a Common Table Expression (CTE) to calculate the average remuneration for different groups.
  WITH Medias AS (
    SELECT 'aposentados_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_aposentados_remuneracao`
   UNION ALL
   SELECT 'pensionista_civil' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_pensionista_remuneracao`
   UNION ALL
   SELECT 'militar_reserva_reforma' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
   FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_reserva_reforma_remuneracao`
   UNION ALL
   SELECT 'pensionista_militar' AS detalhamento, AVG(REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS media
   FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_pensionistas_remuneracao`
  ),

  # Use a second CTE to calculate the median remuneration for the same groups.
  Medianas AS (
  SELECT 'aposentados_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS,
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_aposentados_remuneracao`
  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))

  UNION ALL

  SELECT 'pensionista_civil' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS,
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_siape_pensionista_remuneracao`

  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
  UNION ALL

  SELECT 'militar_reserva_reforma' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS,
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_reserva_reforma_remuneracao`

  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))
  UNION ALL

  SELECT 'pensionista_militar' AS detalhamento,
         IF(MOD(total_count, 2) = 1, REMUNERACAO_BASICA_BRUTA_EM_REAIS,
         (REMUNERACAO_BASICA_BRUTA_EM_REAIS + LAG(REMUNERACAO_BASICA_BRUTA_EM_REAIS, 1) OVER(ORDER BY row_number)) / 2) AS mediana
  FROM (
    SELECT REMUNERACAO_BASICA_BRUTA_EM_REAIS,
           COUNT(REMUNERACAO_BASICA_BRUTA_EM_REAIS) OVER() AS total_count,
           ROW_NUMBER() OVER(ORDER BY REMUNERACAO_BASICA_BRUTA_EM_REAIS) AS row_number
    FROM `repositoriodedadosgpsp.portal_transparencia_cgu.{ano}_nov_militares_pensionistas_remuneracao`

  )
  WHERE row_number IN (FLOOR((total_count + 1) / 2), CEIL((total_count + 1) / 2))


  )

  # Join the results from the Medias and Medianas CTEs.
  SELECT
    M.detalhamento,
  M.media,
  MD.mediana
FROM Medias M
LEFT JOIN Medianas MD
ON M.detalhamento = MD.detalhamento
"""

  # Execute the query for the current year.
  query_job = client.query(query)
  # Create a dynamic DataFrame name based on the year (e.g., "df2020").
  df_name = f"df{ano}"
  # Execute code to create a DataFrame with the dynamic name from the query result.
  exec(f"{df_name} = query_job.to_dataframe()")
# Create a DataFrame for each year.

# Define a list of years from 2021 to 2024 for concatenation.
anos2 = [2021,2022,2023,2024]

# This re-definition is redundant.
anos2 = [2021,2022,2023,2024]
# Manually add an 'ano' column to the 2020 DataFrame.
df2020['ano'] = 2020
# Filter out rows from df2020 where the 'mediana' is null.
df2020= df2020[df2020['mediana'].notna()]

# Loop through the remaining years to combine the DataFrames.
for ano in anos2:
   # Create the variable name for the DataFrame based on the year.
    df_name = f"df{ano}"

    # Retrieve the DataFrame corresponding to the year from the global scope.
    df = globals().get(df_name)

    if df is not None:  # Check if the DataFrame exists.
        df['ano'] = ano  # Add the 'ano' column.
        df = df[df['mediana'].notna()]  # Filter rows where 'mediana' is not NaN.

        # Concatenate the current DataFrame to the main df2020 DataFrame.
        df2020 = pd.concat([df2020, df], ignore_index=True)

# Assign the final combined DataFrame to a new variable 'df'.
df = df2020

# Read inflation (IPCA) data from a CSV file.
ipca = pd.read_csv('ipca_ibge_graf_cgu.csv')

# Ensure the 'ano' column is of integer type for merging.
df['ano'] = df['ano'].astype(int)

# Merge the remuneration data with the inflation data on the 'ano' column.
df1=df.merge(ipca[['ano','fator_correcao']], how='left', on='ano')

# Calculate the inflation-adjusted average remuneration.
df1['media_corrigido']= df1['media']*df1['fator_correcao']
# Calculate the inflation-adjusted median remuneration.
df1['mediana_corrigido']= df1['mediana']*df1['fator_correcao']

# Select and reorder the columns for the final DataFrame.
df1=df1[['ano','detalhamento', 'media', 'mediana','media_corrigido', 'mediana_corrigido','fator_correcao']]

# Create a dictionary to map technical names to more readable descriptions.
mapeamento_nomes = {
    'pensionista_militar': 'Pensão Militar',
    'aposentados_civil': 'Aposentadoria Civil',
    'militar_reserva_reforma': 'Reserva ou Reforma Militar',
    'pensionista_civil': 'Pensão Civil'
}

# Apply the name mapping to the 'detalhamento' column.
df1['detalhamento'] = df1['detalhamento'].replace(mapeamento_nomes)

# Upload

In [ ]:
# Import the bigquery library from google.cloud
from google.cloud import bigquery

# Initialize the BigQuery client, specifying the Google Cloud project ID.
# This client object is the main entry point for interacting with the BigQuery API.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the BigQuery dataset named 'perfil_remuneracao'.
# This object points to the dataset where the table will be created or updated.
dataset_ref = client.dataset('perfil_remuneracao')

# Define the schema for the destination BigQuery table.
# The schema is a list of SchemaField objects, where each object defines a column's:
# 1. Name (e.g., 'ano')
# 2. Data type (e.g., 'INTEGER')
# 3. Description (e.g., 'Ano de referencia da informacao')
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano de referencia da informacao'),
    bigquery.SchemaField('detalhamento', 'STRING', description='Detalhamento do grupo que estamos nos referindo'),
    bigquery.SchemaField('media', 'FLOAT', description='Valor da média sem correção'),
    bigquery.SchemaField('mediana', 'FLOAT', description='Valor da mediana sem correção'),
    bigquery.SchemaField('media_corrigido', 'FLOAT', description='Valor da média com correção'),
    bigquery.SchemaField('mediana_corrigido', 'FLOAT', description='Valor da mediana com correção'),
    bigquery.SchemaField('fator_correcao', 'FLOAT', description='Fator de correção para dezembro de 2023')
]

# Create a reference to the target table within the dataset specified earlier.
# The table will be named 'CGU_remuneracao_aposenta_pensao_v3'.
table_ref = dataset_ref.table('CGU_remuneracao_aposenta_pensao_v3') # table name in the format SOURCE_something_intuitive_data

# Configure the load job by creating a LoadJobConfig object.
# Here, we specify the schema that BigQuery should use for the table. This ensures
# that the columns in BigQuery have the correct data types and descriptions.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start the job to load data from the pandas DataFrame 'df1' into the specified BigQuery table ('table_ref').
# The job is configured with the previously defined 'job_config'. This command sends the data to BigQuery.
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)

# Wait for the load job to complete and retrieve its result.
# This line is blocking and will pause the script's execution until the data upload is finished.
# It's crucial for ensuring the data is fully loaded before the script ends or proceeds.
job.result()